# Purpose

### 2022-02-14
In this notebook I'll select the clusters for the new FPR experiments for Canada, UK, Australia, & India.

Note that this is supposed to be an SFW experiment, so we'll need to filter out subreddits that are `over_18` or rated as `X`.

In one sheet include BOTH subreddit names & subreddit IDs.

---

TODO: Haven't included place logic (e.g., add direction to: city, state, country subreddits.)


### Updates
2022-02-16: 
I [David] will update the QA sheet so that we have standardized columns/format. Otherwise it'll be more work for us to wait for country managers to format things and then standardize them after the fact



# Imports & notebook setup

In [ ]:
%load_ext autoreload
%autoreload 2

# Register bigquery magic (only needed for laptop/local, not colab)
# %load_ext google.cloud.bigquery

In [ ]:
# colab auth for BigQuery
from google.colab import auth, files, drive
auth.authenticate_user()
print('Authenticated')

Authenticated


## Install custom library

### Append google drive path so we can install library from there

In [ ]:
# Attach google drive & import my python utility functions
# if drive.mount() fails, you can also:
#   MANUALLY CLICK ON "Mount Drive"
import sys


g_drive_root = '/content/drive'

try:
    drive.mount(g_drive_root, force_remount=True)
    print('   Authenticated & mounted Google Drive')
    
except Exception as e:
    try:
        drive._mount(g_drive_root, force_remount=True)
        print('   Authenticated & mounted Google Drive')
    except Exception as e:
        print(e)
        raise Exception('You might need to manually mount google drive to colab')

l_paths_to_append = [
    f'{g_drive_root}/MyDrive/Colab Notebooks',

    # need to append the path to subclu so that colab can import things properly
    f'{g_drive_root}/MyDrive/Colab Notebooks/subreddit_clustering_i18n'
]
for path_ in l_paths_to_append:
    if path_ in sys.path:
        sys.path.remove(path_)
    print(f" Appending path: {path_}")
    sys.path.append(path_)

Mounted at /content/drive
   Authenticated & mounted Google Drive
 Appending path: /content/drive/MyDrive/Colab Notebooks
 Appending path: /content/drive/MyDrive/Colab Notebooks/subreddit_clustering_i18n


### Install library

In [ ]:
# install subclu & libraries needed to read parquet files from GCS & spreadsheets
#  make sure to use the [colab] `extra` because it includes colab-specific libraries
module_path = f"{g_drive_root}/MyDrive/Colab Notebooks/subreddit_clustering_i18n/[colab]"

!pip install -e $"$module_path" --quiet

     |████████████████████████████████| 10.1 MB 7.0 MB/s 
     |████████████████████████████████| 14.2 MB 20.1 MB/s 
     |████████████████████████████████| 965 kB 30.5 MB/s 
     |████████████████████████████████| 144 kB 37.3 MB/s 
     |████████████████████████████████| 285 kB 53.7 MB/s 
     |████████████████████████████████| 13.2 MB 36.7 MB/s 
     |████████████████████████████████| 79.9 MB 104 kB/s 
     |████████████████████████████████| 133 kB 50.5 MB/s 
     |████████████████████████████████| 715 kB 54.0 MB/s 
     |████████████████████████████████| 74 kB 3.0 MB/s 
     |████████████████████████████████| 112 kB 43.8 MB/s 
     |████████████████████████████████| 146 kB 40.5 MB/s 
     |████████████████████████████████| 58 kB 4.4 MB/s 
     |████████████████████████████████| 1.1 MB 35.9 MB/s 
     |████████████████████████████████| 180 kB 40.3 MB/s 
     |████████████████████████████████| 79 kB 6.4 MB/s 
     |████████████████████████████████| 596 kB 40.6 MB/s 
     |████████████

## Regular Imports

In [ ]:
import os
from datetime import datetime

from google.cloud import bigquery

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib_venn import venn2_unweighted, venn3_unweighted
from tqdm import tqdm


# auth for google sheets
import gspread
from oauth2client.client import GoogleCredentials


gc = gspread.authorize(GoogleCredentials.get_application_default())

# os.environ['GOOGLE_CLOUD_PROJECT'] = 'data-science-prod-218515'
os.environ['GOOGLE_CLOUD_PROJECT'] = 'data-prod-165221'

[autoreload of cloudpickle failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: cannot import name '_should_pickle_by_reference' from 'cloudpickle.cloudpickle' (/usr/local/lib/python3.7/dist-packages/cloudpickle/cloudpickle.py)
]


## Custom imports

In [ ]:
# subclu imports
import subclu
from subclu.utils.eda import (
    setup_logging, counts_describe, value_counts_and_pcts,
    notebook_display_config, print_lib_versions,
    style_df_numeric, reorder_array,
)
from subclu.models.clustering_utils import (
    create_dynamic_clusters,
    convert_distance_or_ab_to_list_for_fpr,
    reshape_df_to_get_1_cluster_per_row,
    get_primary_topic_mix_cols,
)

from subclu.models.reshape_clusters_v041 import (
    keep_only_target_labels,
    get_table_for_optimal_dynamic_cluster_params,
    get_dynamic_cluster_summary,
)


setup_logging()
notebook_display_config()
print_lib_versions([gspread, pd, np])

python		v 3.7.12
===
gspread		v: 5.1.1
pandas		v: 1.3.5
numpy		v: 1.21.5


# Checklist to re-run for a country:

- change country name in geo-relevance query
- change country name for google sheet name
    - add google sheet KEY (after created)

# Load data from BigQuery

## Load subreddit geo-relevance & cultural relevance metadata

This data is already in bigQuery so read it straight from there. We'll use it to filter out geo-relevant (German) subs.

Also add the latest ratings so that we can filter based on those.

English-speaking countries don't have ambassador subs right now, so we should be able to create a standard template and replace the country name for these queries.

### SQL geo & cultural

In [ ]:
%%time
%%bigquery df_geo_and_lang --project data-science-prod-218515 

-- Select geo+cultural subreddits for a target country
--  And add latest rating & over_18 flags to exclude X-rated & over_18
DECLARE TARGET_COUNTRY STRING DEFAULT 'Australia';


SELECT
    s.* EXCEPT(over_18, pt, verdict) 
    , nt.rating_name
    , nt.primary_topic
    , nt.rating_short
    , slo.over_18
    , slo.allow_discovery
    , CASE 
        WHEN(COALESCE(slo.over_18, 'f') = 't') THEN 'over_18_or_X_M_D_V'
        WHEN(COALESCE(nt.rating_short, '') IN ('X', 'M', 'D', 'V')) THEN 'over_18_or_X_M_D_V'
        ELSE 'unrated_or_E'
    END AS grouped_rating

FROM `reddit-employee-datasets.david_bermejo.subclu_v0041_subreddit_clusters_c_a` AS t
    -- Inner join b/c we only want to keep subs that are geo-relevant AND in topic model
    INNER JOIN (
        SELECT *
        FROM `reddit-employee-datasets.david_bermejo.subclu_subreddit_geo_score_standardized_20220212`
        WHERE country_name = TARGET_COUNTRY
    ) AS s
        ON t.subreddit_id = s.subreddit_id

    -- Add rating so we can get an estimate for how many we can actually use for recommendation
    LEFT JOIN (
        SELECT *
        FROM `data-prod-165221.ds_v2_postgres_tables.subreddit_lookup`
        -- Get latest partition
        WHERE dt = DATE(CURRENT_DATE() - 2)
    ) AS slo
    ON s.subreddit_id = slo.subreddit_id
    LEFT JOIN (
        SELECT * FROM `data-prod-165221.cnc.shredded_crowdsource_topic_and_rating`
        WHERE pt = DATE(CURRENT_DATE() - 2)
    ) AS nt
        ON s.subreddit_id = nt.subreddit_id

    -- Exclude popular US subreddits
    -- Can't query this table from local notebook because of errors getting google drive permissions. smh, excludefor now
    LEFT JOIN `reddit-employee-datasets.david_bermejo.subclu_subreddits_top_us_to_exclude_from_relevance` tus
        ON s.subreddit_name = LOWER(tus.subreddit_name)

WHERE 1=1
    AND s.subreddit_name != 'profile'
    AND COALESCE(s.type, '') = 'public'
    AND COALESCE(s.verdict, 'f') <> 'admin_removed'
    AND COALESCE(slo.over_18, 'f') = 'f'
    AND COALESCE(nt.rating_short, '') NOT IN ('X', 'D')

    AND(
        s.geo_relevance_default = TRUE
        OR s.relevance_percent_by_subreddit = TRUE
        OR s.relevance_percent_by_country_standardized = TRUE
    )
    AND country_name IN (
            TARGET_COUNTRY
        )

    AND (
         -- Exclude subs that are top in US but we want to exclude as culturally relevant
         --  For simplicity, let's go with the English exclusion (more relaxed) than the non-English one
         COALESCE(tus.english_exclude_from_relevance, '') <> 'exclude'
    )

ORDER BY e_users_percent_by_country_standardized DESC, users_l7 DESC, subreddit_name
;

CPU times: user 592 ms, sys: 57.2 ms, total: 649 ms
Wall time: 12.4 s


### Check df with geo + language information

In [ ]:
print(df_geo_and_lang.shape)

(1419, 26)


In [ ]:
df_geo_and_lang.iloc[:4, :9]

,subreddit_id,subreddit_name,country_name,geo_relevance_default,b_users_percent_by_subreddit,e_users_percent_by_country_standardized,c_users_percent_by_country,d_users_percent_by_country_rank,relevance_percent_by_subreddit
0,t5_2qkhb,melbourne,Australia,True,0.669184,10.780123,0.029058,3,True
1,t5_2qkob,sydney,Australia,True,0.821898,10.706480,0.019942,10,True
2,t5_2uo3q,ausfinance,Australia,True,0.831555,10.663609,0.018428,14,True
3,t5_2qh8e,australia,Australia,True,0.414442,10.460037,0.046857,2,True


## Load model labels (clusters)

The clusters now live in a Big Query table and have standardized names, so pull the data from there.

### SQL labels


In [ ]:
%%time
%%bigquery df_labels --project data-science-prod-218515 

-- select subreddit clusters from bigQuery

SELECT
    sc.subreddit_id
    , sc.subreddit_name
    , nt.primary_topic

    , sc.* EXCEPT(subreddit_id, subreddit_name, primary_topic_1214)
FROM `reddit-employee-datasets.david_bermejo.subclu_v0041_subreddit_clusters_c_a` sc
    LEFT JOIN (
        -- New view should be visible to all, but still comes from cnc_taxonomy_cassandra_sync
        SELECT * FROM `data-prod-165221.cnc.shredded_crowdsource_topic_and_rating`
        WHERE DATE(pt) = (CURRENT_DATE() - 2)
    ) AS nt
        ON sc.subreddit_id = nt.subreddit_id
;

CPU times: user 9.78 s, sys: 574 ms, total: 10.4 s
Wall time: 26.9 s


### Check label outputs

In [ ]:
print(df_labels.shape)
df_labels.iloc[:4, :9]

(49558, 51)


,subreddit_id,subreddit_name,primary_topic,model_sort_order,posts_for_modeling_count,k_0013_label,k_0023_label,k_0041_label,k_0059_label
0,t5_5a9iie,progonlydj,None,40079,1000,12,19,34,49
1,t5_2x9c7,googleplaymusic,Music,40080,31,12,19,34,49
2,t5_3jzsk,ravedj,Music,40081,1000,12,19,34,49
3,t5_2rgie,happyhardcore,Music,40082,152,12,19,34,49


In [ ]:
counts_describe(df_labels.iloc[:, :9])

,dtype,count,unique,unique-percent,null-count,null-percent
subreddit_id,object,"49,558","49,558",100.00%,0,0.00%
subreddit_name,object,"49,558","49,558",100.00%,0,0.00%
primary_topic,object,"40,714",52,0.13%,"8,844",17.85%
model_sort_order,int64,"49,558","49,558",100.00%,0,0.00%
posts_for_modeling_count,int64,"49,558",999,2.02%,0,0.00%
k_0013_label,int64,"49,558",13,0.03%,0,0.00%
k_0023_label,int64,"49,558",23,0.05%,0,0.00%
k_0041_label,int64,"49,558",41,0.08%,0,0.00%
k_0059_label,int64,"49,558",59,0.12%,0,0.00%


# Reshape data
Apply reshaping fxns so that we can export the data in a format that's good for QA.

## Keep only labels for Target subreddits


In [ ]:
%%time
df_labels_target = keep_only_target_labels(
    df_labels=df_labels,
    df_geo=df_geo_and_lang,
    col_sort_order='model_sort_order',
    l_ix_subs=['subreddit_id', 'subreddit_name'],
    l_cols_to_front=None,
    geo_cols_to_drop=None,
)

0 <- subs to drop b/c they're not in model
(1419, 74) <- df_labels_target.shape
CPU times: user 271 ms, sys: 1.69 ms, total: 273 ms
Wall time: 274 ms


In [ ]:
counts_describe(df_labels_target.iloc[:, :15])

,dtype,count,unique,unique-percent,null-count,null-percent
model_sort_order,int64,"1,419","1,419",100.00%,0,0.00%
subreddit_id,object,"1,419","1,419",100.00%,0,0.00%
subreddit_name,object,"1,419","1,419",100.00%,0,0.00%
primary_topic,object,"1,226",51,4.16%,193,13.60%
rating_short,object,"1,378",3,0.22%,41,2.89%
over_18,object,326,1,0.31%,"1,093",77.03%
rating_name,object,"1,378",3,0.22%,41,2.89%
posts_for_modeling_count,int64,"1,419",512,36.08%,0,0.00%
k_0013_label,int64,"1,419",12,0.85%,0,0.00%
k_0023_label,int64,"1,419",22,1.55%,0,0.00%


## Run loop to find "optimal" min_num of subreddits for dynamic clusters


We want to balance two things:
- prevent orphan subreddits
- prevent clusters that are too large to be meaningful

In order to do this at a country level, we'll be better off starting with smallest cluster size and roll up until we have at least N subreddits in one cluster.

Find optimal `min_subreddits_in_cluster` based on:
- `orphan count`, 
- `number of clusters`,
- & other info

number might be different for each country and even within a country it might differ by when we filter NSFW subs.

### Loop

In [ ]:
%%time

col_new_cluster_val = 'cluster_label'
col_new_cluster_name = 'cluster_label_k'
col_new_cluster_prim_topic = 'cluster_majority_primary_topic'
col_new_cluster_topic_mix = 'cluster_topic_mix'

df_optimal_min_check = get_table_for_optimal_dynamic_cluster_params(
        df_labels_target=df_labels_target,
        col_new_cluster_val=col_new_cluster_val,
        col_new_cluster_name=col_new_cluster_name,
        col_new_cluster_prim_topic=col_new_cluster_prim_topic,
        col_new_cluster_topic_mix=col_new_cluster_topic_mix,
        min_subs_in_cluster_list=np.arange(3, 11),
        verbose=False,
)

100%|██████████| 16/16 [00:00<00:00, 18.88it/s]

100%|██████████| 16/16 [00:00<00:00, 19.99it/s]

100%|██████████| 16/16 [00:00<00:00, 18.56it/s]

100%|██████████| 16/16 [00:00<00:00, 18.23it/s]

100%|██████████| 16/16 [00:00<00:00, 18.31it/s]

100%|██████████| 16/16 [00:00<00:00, 18.94it/s]

100%|██████████| 16/16 [00:00<00:00, 20.74it/s]

100%|██████████| 16/16 [00:00<00:00, 19.79it/s]

100%|██████████| 8/8 [00:12<00:00,  1.57s/it]


CPU times: user 12.2 s, sys: 262 ms, total: 12.4 s
Wall time: 12.7 s


### Display loop results

In [ ]:
def highlight_below_threshold(val, threshold=1):
    if val <= threshold:
        return "color:purple; font-weight: bold; background-color:yellow;"
    else:
        return ''

col_num_orph_subs = 'num_orphan_subreddits'
# col_num_subs_mean = 'num_subreddits_per_cluster_mean'
col_num_subs_median = 'num_subreddits_per_cluster_median'

style_df_numeric(
    df_optimal_min_check,
    rename_cols_for_display=True,
    l_bar_simple=[col_num_orph_subs,
                  col_num_subs_median,]
).applymap(highlight_below_threshold, subset=[col_num_orph_subs.replace('_', ' ')])


,subs to cluster count,min subreddits in cluster,cluster count,num orphan subreddits,num subreddits per cluster mean,num subreddits per cluster median,num clusters with mature primary topic,cluster ids with orphans
0,"1,419",3,276,3,5.14,5.00,20,"0001, 0007, 0009"
1,"1,419",4,228,3,6.22,6.00,16,"0007, 0009, 0010"
2,"1,419",5,190,1,7.47,7.00,16,0011
3,"1,419",6,165,1,8.60,8.00,12,0011
4,"1,419",7,139,1,10.21,9.00,12,0009
5,"1,419",8,121,1,11.73,11.00,11,0010
6,"1,419",9,111,0,12.78,12.00,9,
7,"1,419",10,103,1,13.78,13.00,9,0007


In [ ]:
del df_optimal_min_check

## Get dyanimc clusters (apply optimal num from above)

side bar: about 57% of subreddits in Australia only had a single primary topic as their `topic_mix`, so combining `primary topic` might not give us as much info as we hoped.

At the same time, for 43% of subs we might get additional detail by combining the primary topics.


In [ ]:
n_min_subs_in_cluster_optimal = 5
n_mix_start = 4
l_ix = ['subreddit_id', 'subreddit_name']
col_new_cluster_topic_mix = 'cluster_topic_mix'
col_subreddit_topic_mix = 'subreddit_full_topic_mix'
col_full_depth_mix_count = 'subreddit_full_topic_mix_count'
suffix_new_topic_mix = '_topic_mix_nested'

# even if cluster at k < 20 is generic, keep it to avoid orphan subs
l_cols_labels = (
    [c for c in df_labels_target.columns
        if all([c != col_new_cluster_val, c.endswith('_label')])
        ]
)

df_labels_target_dynamic_raw = create_dynamic_clusters(
    df_labels_target,
    agg_strategy='aggregate_small_clusters',
    min_subreddits_in_cluster=5,
    l_cols_labels_input=l_cols_labels,
    col_new_cluster_val=col_new_cluster_val,
    col_new_cluster_name=col_new_cluster_name,
    col_new_cluster_prim_topic=col_new_cluster_prim_topic,
    n_mix_start=n_mix_start,
    col_new_cluster_topic_mix=col_new_cluster_topic_mix,
    col_subreddit_topic_mix=col_subreddit_topic_mix,
    col_full_depth_mix_count=col_full_depth_mix_count,
    suffix_new_topic_mix=suffix_new_topic_mix,
    l_ix=l_ix,
    verbose=True,
)

00:24:06 | INFO | "Concat'ing nested cluster labels..."
00:24:06 | INFO | "Getting topic mix at different depths..."
00:24:06 | INFO | "  Assigning base topic mix cols"
00:24:06 | INFO | "  Creating deepest base topic mix col..."
00:24:06 | INFO | "  Iterating through additional subs with multiple topics..."
100%|██████████| 16/16 [00:00<00:00, 20.03it/s]
00:24:07 | INFO | "Initializing values for strategy: aggregate_small_clusters"
00:24:07 | INFO | "  Looping to roll-up clusters from smallest to largest..."
100%|██████████| 21/21 [00:00<00:00, 93.85it/s]
00:24:07 | INFO | "(1419, 124) <- output shape"


In [ ]:
# [c for c in df_labels_target_dynamic_raw.columns if 'nested' in c]

In [ ]:
style_df_numeric(
    get_dynamic_cluster_summary(
        df_labels_target_dynamic_raw,
        return_dict=False,
    ),
    rename_cols_for_display=True,
)

,cluster count,num orphan subreddits,num subreddits per cluster mean,num subreddits per cluster median,num clusters with mature primary topic,cluster ids with orphans
0,190,1,7.47,7.00,16,0011


In [ ]:
# # check column order
style_df_numeric(
    df_labels_target_dynamic_raw.iloc[104:109, :15],
    rename_cols_for_display=True,
    int_labels=['total_users_in', 'num_of_countries_', 'users_in_subreddit_from_country_l28',
                    'by_country_rank',
                    ],
    pct_cols=['b_users_percent_by_subreddit',
                  'c_users_percent_by_country',
                  'users_percent_by_country_avg',
                  ],
    pct_labels='',
)

,subreddit id,subreddit name,cluster topic mix,primary topic,rating short,subreddit full topic mix,rating name,over 18,geo relevance default,relevance percent by subreddit,relevance percent by country standardized,b users percent by subreddit,e users percent by country standardized,d users percent by country rank,model sort order
104,t5_38cev,melbournecycling,Cars and Motor Vehicles | Hobbies,-,E,Cars and Motor Vehicles | Hobbies,Everyone,-,False,True,True,84.24%,2.04,"24,356","18,028"
105,t5_2wy6u,ausbike,Cars and Motor Vehicles | Hobbies,Fitness and Nutrition,E,Cars and Motor Vehicles | Hobbies,Everyone,-,True,True,True,78.96%,5.01,"5,424","18,029"
106,t5_2tbmq,bikecommuting,Cars and Motor Vehicles | Hobbies,Travel,E,Cars and Motor Vehicles | Hobbies,Everyone,-,False,False,True,4.84%,2.33,"4,230","18,032"
107,t5_2qhyi,cycling,Cars and Motor Vehicles | Hobbies,Sports,E,Cars and Motor Vehicles | Hobbies,Everyone,f,False,False,True,5.94%,2.47,955,"18,033"
108,t5_32hd6,electricskateboarding,Cars and Motor Vehicles,Hobbies,E,Cars and Motor Vehicles,Everyone,-,False,False,True,6.85%,2.28,"5,976","18,037"


### Minor QA checks

In [ ]:
# # check column order
# style_df_numeric(
#     df_labels_target_dynamic_raw.iloc[70:74, -22:],
#     rename_cols_for_display=True,
#     int_labels=['total_users_in', 'num_of_countries_', 'users_in_subreddit_from_country_l28',
#                     'by_country_rank',
#                     ],
#     pct_cols=['b_users_percent_by_subreddit',
#                   'c_users_percent_by_country',
#                   'users_percent_by_country_avg',
#                   ],
#     pct_labels='',
# )

In [ ]:
value_counts_and_pcts(
    df_labels_target_dynamic_raw[col_new_cluster_topic_mix],
    top_n=9,
)

,cluster_topic_mix-count,cluster_topic_mix-percent,cluster_topic_mix-pct_cumulative_sum
Mature Themes and Adult Content,92,6.5%,6.5%
Gaming,81,5.7%,12.2%
Television | Podcasts and Streamers,76,5.4%,17.5%
Animals and Pets,69,4.9%,22.4%
Medical and Mental Health,69,4.9%,27.3%
Music,59,4.2%,31.4%
Cars and Motor Vehicles,55,3.9%,35.3%
Television,55,3.9%,39.2%
Sports,49,3.5%,42.6%


In [ ]:
# how many final clusters have multiple topics?
value_counts_and_pcts(
    df_labels_target_dynamic_raw[col_new_cluster_topic_mix].str.count('\|')
)

,cluster_topic_mix-count,cluster_topic_mix-percent,cluster_topic_mix-pct_cumulative_sum
0,873,61.5%,61.5%
1,385,27.1%,88.7%
2,135,9.5%,98.2%
3,26,1.8%,100.0%


In [ ]:
# how many SUBREDDITS have multiple topics? (when we check the deepest clusters)
#  these two calls are equivalent

# value_counts_and_pcts(
#     df_labels_target_dynamic_raw[col_subreddit_topic_mix].str.count('\|')
# )

value_counts_and_pcts(
    df_labels_target_dynamic_raw[col_full_depth_mix_count]
)

,subreddit_full_topic_mix_count-count,subreddit_full_topic_mix_count-percent,subreddit_full_topic_mix_count-pct_cumulative_sum
1,652,45.9%,45.9%
2,433,30.5%,76.5%
3,228,16.1%,92.5%
4,84,5.9%,98.4%
5,21,1.5%,99.9%
6,1,0.1%,100.0%


In [ ]:
style_df_numeric(
    df_labels_target_dynamic_raw
    [df_labels_target_dynamic_raw[col_full_depth_mix_count] >= 5]
    .iloc[-5:, :9]
    ,
    rename_cols_for_display=True,
)

,subreddit id,subreddit name,cluster topic mix,primary topic,rating short,subreddit full topic mix,rating name,over 18,geo relevance default
884,t5_3ej5o,australiatravel,"History | Place | Mature Themes and Adult Content | Culture, Race, and Ethnicity",-,-,"History | Place | Mature Themes and Adult Content | Culture, Race, and Ethnicity | Internet Culture and Memes",-,-,True
885,t5_56fwff,australiacommercial,"History | Place | Mature Themes and Adult Content | Culture, Race, and Ethnicity",-,-,"History | Place | Mature Themes and Adult Content | Culture, Race, and Ethnicity | Internet Culture and Memes",-,-,True
886,t5_2x177,ameristralia,"History | Place | Mature Themes and Adult Content | Culture, Race, and Ethnicity",Internet Culture and Memes,E,"History | Place | Mature Themes and Adult Content | Culture, Race, and Ethnicity | Internet Culture and Memes",Everyone,-,True
887,t5_340dk,askanaustralian,"History | Place | Mature Themes and Adult Content | Culture, Race, and Ethnicity",Learning and Education,E,"History | Place | Mature Themes and Adult Content | Culture, Race, and Ethnicity | Internet Culture and Memes",Everyone,-,True
1402,t5_2s5sk,lv426,Art,Movies,E,Art | Gaming | Television | Internet Culture and Memes | Movies,Everyone,-,False


## Re-assign orphan subreddits (optional)

If there are subreddits that are orphan (see summary above), check them out to see if we can re-assign them w/o too much work. if we can't skip and move to the next country.

In [ ]:
# check subs around orphan sub
# n_plus_minus_ = 5
ix_orphan_ = (
    df_labels_target_dynamic_raw
    [df_labels_target_dynamic_raw[col_new_cluster_val] == '0011']
    .index
)
df_labels_target_dynamic_raw.iloc[ix_orphan_, :9]

,subreddit_id,subreddit_name,cluster_topic_mix,primary_topic,rating_short,subreddit_full_topic_mix,rating_name,over_18,geo_relevance_default
930,t5_2tt7r,realms,Gaming,Gaming,E,Gaming,Everyone,None,False


In [ ]:
# check other subs that are in the same cluster as orphan sub (at broadest level)
l_cols_orphan_check = (
    [
        'subreddit_id',
        col_new_cluster_topic_mix, 
        # col_new_cluster_val,  # this can be really long and makes comparing harder
        # col_subreddit_topic_mix,
        'subreddit_name', 
        col_new_cluster_name
    ] +
    l_cols_labels[:-5]
)

style_df_numeric(
    df_labels_target_dynamic_raw
    [df_labels_target_dynamic_raw['k_0013_label'] == 11]
    [l_cols_orphan_check]
    .iloc[3:14, :50]
    ,
    l_bar_simple=[c for c in l_cols_orphan_check[4:] if c.endswith('_label')],
    rename_cols_for_display=True,

)

,subreddit id,cluster topic mix,subreddit name,cluster label k,k 0013 label,k 0023 label,k 0041 label,k 0059 label,k 0063 label,k 0079 label,k 0085 label,k 0118 label,k 0320 label,k 0657 label,k 0958 label,k 1065 label,k 1560 label,k 1840 label,k 2207 label,k 2351 label,k 2830 label
926,t5_35qsx,Gaming,crowfall,k_0320_label,11,18,32,43,46,58,62,87,244,488,707,789,"1,147","1,348","1,616","1,726","2,066"
927,t5_2ais35,Gaming,destiny2builds,k_0320_label,11,18,32,43,46,58,62,87,244,489,710,792,"1,151","1,355","1,624","1,734","2,074"
928,t5_389nk,Gaming,sharditkeepit,k_0320_label,11,18,32,43,46,58,62,87,244,489,710,792,"1,151","1,355","1,624","1,734","2,074"
929,t5_3os9l4,Gaming,crucibleguidebook,k_0320_label,11,18,32,43,46,58,62,87,244,489,710,792,"1,151","1,355","1,624","1,734","2,074"
930,t5_2tt7r,Gaming,realms,k_0013_label,11,18,32,43,46,58,62,87,246,493,716,799,"1,159","1,363","1,633","1,743","2,086"
931,t5_3nqdi,Gaming,swlegion,k_0085_label,11,18,32,43,46,59,63,88,247,494,718,801,"1,161","1,365","1,635","1,745","2,089"
932,t5_293c3c,Gaming,printedwarhammer,k_0085_label,11,18,32,43,46,59,63,88,247,494,719,802,"1,162","1,367","1,638","1,748","2,092"
933,t5_2y5lg,Gaming,tau40k,k_0085_label,11,18,32,43,46,59,63,88,247,494,719,802,"1,163","1,368","1,639","1,749","2,093"
934,t5_2scss,Gaming,minipainting,k_0085_label,11,18,32,43,46,59,63,88,247,494,719,802,"1,163","1,368","1,639","1,749","2,093"
935,t5_2qnwk,Gaming,battletech,k_0085_label,11,18,32,43,46,59,63,88,247,494,719,802,"1,163","1,368","1,639","1,749","2,094"


In [ ]:
label_k_to_reassign_ = 'k_0320_label'
label_val_to_reassign_ = '0011-0018-0032-0043-0046-0058-0062-0087-0244'
subreddit_id_orphan_ = 't5_2tt7r'

mask_orphan_and_new_group = (
    (df_labels_target_dynamic_raw['subreddit_id'] == subreddit_id_orphan_) |
    (
        (df_labels_target_dynamic_raw[col_new_cluster_name] == label_k_to_reassign_) &
        (df_labels_target_dynamic_raw[col_new_cluster_val] == label_val_to_reassign_)
    )
)

# assign is similar to what we do in the dynamic function
label_k_new_ = 'k_0118_label'
label_val_new_col_ = f"{label_k_new_}_nested"
new_prim_topic_col_ = label_k_new_.replace('_label', '_majority_primary_topic')
c_update_topic_mix_ = label_k_new_.replace('_label', suffix_new_topic_mix)

df_labels_target_dynamic_raw.loc[
    mask_orphan_and_new_group,
    col_new_cluster_name
] = label_k_new_

df_labels_target_dynamic_raw.loc[
    mask_orphan_and_new_group,
    col_new_cluster_val
] = df_labels_target_dynamic_raw[mask_orphan_and_new_group][label_val_new_col_]

df_labels_target_dynamic_raw.loc[
    mask_orphan_and_new_group,
    col_new_cluster_prim_topic
] = df_labels_target_dynamic_raw[mask_orphan_and_new_group][new_prim_topic_col_]

df_labels_target_dynamic_raw.loc[
    mask_orphan_and_new_group,
    col_new_cluster_topic_mix
] = df_labels_target_dynamic_raw[mask_orphan_and_new_group][c_update_topic_mix_]

del mask_orphan_and_new_group, label_k_to_reassign_, label_val_to_reassign_
del label_k_new_, label_val_new_col_, new_prim_topic_col_

In [ ]:
# check again, num of orphans should be lower than before
style_df_numeric(
    get_dynamic_cluster_summary(
        df_labels_target_dynamic_raw,
        return_dict=False,
    ),
    rename_cols_for_display=True,
)

,cluster count,num orphan subreddits,num subreddits per cluster mean,num subreddits per cluster median,num clusters with mature primary topic,cluster ids with orphans
0,189,0,7.51,7.00,16,


In [ ]:
value_counts_and_pcts(
    df_labels_target_dynamic_raw,
    ['cluster_label'],
    top_n=None,
    return_df=True
)['count'].describe()

count    189.000000
mean       7.507937
std        2.402612
min        3.000000
25%        6.000000
50%        7.000000
75%        8.000000
max       22.000000
Name: count, dtype: float64

## Get cluster for humans (list of subs in a cluster in a cell)
Here we get 1 cluster per row. 
Use cases:
- It makes it easier to quickly check NSFW clusters that we'll filter out
- we'll append the list of subreddit names from here to the final table for QA (makes it easier to evaluate whether the cluster makes sense).


In [ ]:
col_subs_in_cluster_count = 'subs_in_cluster_count'
col_list_cluster_names = 'list_cluster_subreddit_names'

df_cluster_for_humans = reshape_df_to_get_1_cluster_per_row(
    df_labels_target_dynamic_raw,
    col_counterpart_count=col_subs_in_cluster_count,
    col_list_cluster_names=col_list_cluster_names,
    col_list_cluster_ids='list_cluster_subreddit_ids',
    col_new_cluster_val=col_new_cluster_val,
    col_new_cluster_name=col_new_cluster_name,
    col_new_cluster_topic=col_new_cluster_topic_mix,
    verbose=False,
    get_one_column_per_sub_id=False,
)

(189, 6)


In [ ]:
df_cluster_for_humans.iloc[40:45, 1:]

,cluster_label_k,cluster_topic_mix,subs_in_cluster_count,list_cluster_subreddit_ids,list_cluster_subreddit_names
40,k_1065_label,Hobbies | Crafts and DIY,8,"t5_2rpor, t5_2rjcg, t5_2sczp, t5_hu6et, t5_315b8, t5_2srbo, t5_ie6pz, t5_2r57p","crossstitch, quilting, sewing, craftsnark, vintagesewing, sewhelp, punchneedle, knots"
41,k_3927_label,Hobbies | Crafts and DIY | Art,6,"t5_2re38, t5_2sk72, t5_2sc8d, t5_2yqob, t5_2um9k, t5_2s48h","pottery, ceramics, resin, resincasting, clay, polymerclay"
42,k_2351_label,Hobbies | Crafts and DIY | Fashion,7,"t5_2qkpi, t5_2s8y8, t5_2yhx0, t5_2rq1q, t5_2rv093, t5_2s2hj6, t5_35368","jewelry, jewelers, opals, diamonds, labdiamond, moissanitebst, moissanite"
43,k_0085_label,Hobbies | Beauty and Makeup,8,"t5_2qlac, t5_310nr, t5_2w8pb, t5_4yrxf5, t5_sb9wh, t5_2qqe7, t5_3iftm, t5_3a04j","beauty, ausskincare, indiemakeupandmore, fragranceaustralia, aussiemakeuptrade, mullets, eyelashextensions, equalattraction"
44,k_1560_label,Hobbies | Beauty and Makeup,7,"t5_2rww2, t5_vuqjc, t5_2qrwc, t5_3ca3n, t5_32g1x, t5_342em, t5_2ys2j","makeupaddiction, makeuplounge, makeup, olivemua, makeuprehab, muacjdiscussion, australianmakeup"


In [ ]:
# df_cluster_for_humans.tail(9)

### Check subs that have mature topics

Make a list of sub IDs to exclude for clean df

In [ ]:
mask_mature_clusters_ = (
    df_cluster_for_humans[col_new_cluster_topic_mix].str.lower()
    .str.contains('mature')
)
print(mask_mature_clusters_.sum())

22


In [ ]:
(
    df_cluster_for_humans
    [mask_mature_clusters_]
    .iloc[:11, :]
)

,cluster_label,cluster_label_k,cluster_topic_mix,subs_in_cluster_count,list_cluster_subreddit_ids,list_cluster_subreddit_names
0,0001,k_0013_label,Mature Themes and Adult Content,5,"t5_44faux, t5_3agvg, t5_24v66b, t5_5fbvny, t5_2zui0","neighboursbabez, stephclairesmith, isabelleclarke, sophadophaa, eugeniebouchard"
1,0001-0001-0001-0001-0001-0002-0002-0002,k_0118_label,Mature Themes and Adult Content,6,"t5_iqt8v, t5_3g9c8, t5_2bfy1d, t5_39o6d, t5_28i70j, t5_3fruf","altladyboners, rtgirls, neighboursbabes, aussiebabes, downundercelebs, girlstennis"
2,0001-0001-0002-0002-0002-0003-0003-0003,k_0118_label,Mature Themes and Adult Content,9,"t5_4dgbcr, t5_5bz2ml, t5_4317um, t5_2v07n, t5_l7fuf, t5_547c6n, t5_3g9m96, t5_5clik0, t5_2ixk05","laetitiabrown, polyslartz, jadetunchy, realbikinis, indithew, nelphelanfan, sinoritaee, coffeysisters69, hannahorval"
3,0001-0001-0003-0003-0003-0004-0005-0005,k_0118_label,Mature Themes and Adult Content,7,"t5_25ejbo, t5_4wpiey, t5_4ote3s, t5_4xu6ij, t5_5dhfkc, t5_5bxvct, t5_514f58","hailiedeegan19, oliviadeeble_, forthaboys, saraboyd, australianbabes, alannapow, chanelphelan"
4,0002,k_0013_label,Mature Themes and Adult Content,4,"t5_4qcdkx, t5_48hmss, t5_4nnaei, t5_3zxen1","jacksonnmaddyof, goldcoastsugar, ellactrical, heidihudson"
5,0002-0002,k_0023_label,Mature Themes and Adult Content,6,"t5_4zl04q, t5_4zn255, t5_3xnod0, t5_4056vz, t5_2nl0s9, t5_4chanv","sydney_may2, polyleaked, siarramay, siamayl, ilsawatkins, polygirlsgonewild"
6,0002-0002-0005-0005-0005-0007-0008-0009,k_0118_label,Mature Themes and Adult Content,8,"t5_4cjiib, t5_3kwdmq, t5_4jrkfm, t5_39qqu, t5_3aewyu, t5_4z75gq, t5_2w83vq, t5_4jgdp9","vanessasierra_, jessicathoday, fijiporn, rosievan, luciejaid2, viralvideoleaks, deminovak, zimaanderson"
7,0004,k_0013_label,Mature Themes and Adult Content,4,"t5_55cv32, t5_24s9xa, t5_2wip6, t5_38ezr","bdsmaustralia, brisbanesocial, brisbanegaybros, amwfdating"
8,0005-0007-0012,k_0041_label,Mature Themes and Adult Content,8,"t5_296554, t5_333yu, t5_3jd26q, t5_53z0jy, t5_2858fg, t5_378b57, t5_2n2cey, t5_2qtjz","aussiefeet, trollingforababy, honeybirdette, hookupsperth, communalshowers, aussienaturism, blueycirclejerk, images"
59,0008-0013,k_0023_label,Mature Themes and Adult Content,6,"t5_2qlq6, t5_2qspe, t5_34cyw, t5_2qpe9, t5_2vfzu, t5_2qu5n","audible, etymology, datingoverthirty, onlinedating, thegirlsurvivalguide, polyamory"


In [ ]:
(
    df_cluster_for_humans
    [mask_mature_clusters_]
    .iloc[-12:, :]
)

,cluster_label,cluster_label_k,cluster_topic_mix,subs_in_cluster_count,list_cluster_subreddit_ids,list_cluster_subreddit_names
70,0008-0013-0023-0032-0034-0044-0046-0064,k_0118_label,Mature Themes and Adult Content | Gender,9,"t5_mwfyw, t5_2tnmd, t5_2r1c3, t5_2rfyw, t5_33rcf, t5_30c2m, t5_2rvxp, t5_xaiot, t5_2qkeh","latebloomerlesbians, lgbtaustralia, genderqueer, asianamerican, hapas, asianmasculinity, niceguys, femaledatingstrategy, answers"
71,0008-0013-0023-0032-0034-0044-0046-0064-0181-0356-0518-0580,k_1065_label,Mature Themes and Adult Content | Gender,6,"t5_2t187, t5_2ub9j, t5_2r4b9, t5_3ijj6, t5_31hoq, t5_32viw","mypartneristrans, mtf, asktransgender, transytalk, ask_transgender, transgenderau"
72,0008-0013-0023-0032-0034-0044-0046-0064-0184,k_0320_label,Mature Themes and Adult Content | Gender,7,"t5_3531l, t5_35hao, t5_2r4eo, t5_3hcwf, t5_2xinb, t5_296zi1, t5_39xf0","bumble, hingeapp, ama, aftertheloop, outoftheloop, smolbeansnark, blogsnark"
73,0008-0013-0023-0032-0034-0044-0046-0064-0184-0362-0525-0588,k_1065_label,Mature Themes and Adult Content | Gender | Family and Relationships,8,"t5_2yr9y, t5_nhaha, t5_2vplf, t5_2qhtr, t5_2rv3t, t5_2ug26, t5_11080t, t5_2ra25","bridezillas, weddingshaming, weddingsunder10k, wedding, weddingplanning, engaged, waiting_to_wed, engagementrings"
81,0008-0014-0025-0034-0036-0046-0048-0066,k_0118_label,Mature Themes and Adult Content,9,"t5_31mz6, t5_vt3kk, t5_3jth5, t5_2tkn9, t5_lx01k, t5_3c7rbf, t5_2wgs2u, t5_513pq7, t5_2r9bh","lexapro, mirtazapine_remeron, pristiq, modafinil, sarmssourcetalk, aulean, benzosaus, chemicalmagicau, acid"
82,0008-0014-0025-0034-0036-0046-0048-0066-0190-0378,k_0657_label,Mature Themes and Adult Content,6,"t5_2uggx, t5_2xfv5, t5_2qlxl, t5_2te5i, t5_2scuv, t5_2vu9j","druggardening, poppyseed, mescaline, kava, nitrous, nitrousoxide"
83,0008-0014-0025-0034-0036-0047-0049-0067,k_0118_label,Mature Themes and Adult Content,6,"t5_2t0if, t5_2t7u5, t5_2qpco, t5_3lftlc, t5_43or0d, t5_2ojf76","ausbeer, showerbeer, cocktails, auscann, medicalcannabisoz, medicalcannabisaus"
84,0008-0014-0025-0034-0036-0047-0049-0067-0193,k_0320_label,Mature Themes and Adult Content,8,"t5_2slm7, t5_2tdnb, t5_2tmn1, t5_2tt9m, t5_j3ovh, t5_2u7ap, t5_2xo3j, t5_3a4n7","stonerengineering, ausents, adelaidetrees, melbents, gamersupps, incense, aussievapers, craftymighty"
109,0010-0017-0030-0040-0042-0053-0056-0078-0219-0437-0629-0704-1016-1194-1437-1529-1824-2028-2191-2375-2476-2510,k_3927_label,Place | Mature Themes and Adult Content | Law,6,"t5_2qmsc, t5_2wvvc, t5_2s5e8, t5_38ve9, t5_2gmg9b, t5_4r6wtg","unsolvedmysteries, unresolvedmysteries, truecrime, truecrimediscussion, crimeplus, truecrimemystery"
110,0010-0017-0030-0040-0042-0053-0056-0078-0219-0437-0630-0705,k_1065_label,Place | Mature Themes and Adult Content | Law,7,"t5_3cfj2, t5_iioxt, t5_navml, t5_n9g9j, t5_2cfkv9, t5_1lruuu, t5_fl4tn","earons, mrcruel, chriswatts, shannanwatts, epsteinandfriends, epstein, duggarssnark"


- model did a good job identifying this drug cluster... but yeah def don't want to increase their reach   
    - `0008-0014-0025-0034-0036-0046-0048-0066`
    - `0008-0014-0025-0034-0036-0046-0048-0066-0190-0378`

- the drinking might be SFW... but prob not the 420
    - `0008-0014-0025-0034-0036-0047-0049-0067`
    - `0008-0014-0025-0034-0036-0047-0049-0067-0193`

In [ ]:
l_mature_clusters_to_exclude_from_qa = [
    '0001',
    '0001-0001-0001-0001-0001-0002-0002-0002',
    '0001-0001-0002-0002-0002-0003-0003-0003',
    '0001-0001-0003-0003-0003-0004-0005-0005',
    '0002',
    '0002-0002',
    '0002-0002-0005-0005-0005-0007-0008-0009',
    '0004',
    '0005-0007-0012',
    '0008-0013',  # thegirlsurvivalguide could be good to show, but prob not for ppl looking at r/onlinedating...?
    '0008-0013-0023-0032-0034-0044-0046-0064',  # can of worms...
    '0008-0014-0025-0034-0036-0046-0048-0066',
    '0008-0014-0025-0034-0036-0046-0048-0066-0190-0378',
    '0008-0014-0025-0034-0036-0047-0049-0067',
    '0008-0014-0025-0034-0036-0047-0049-0067-0193',
    '0010-0017-0030-0040-0042-0053-0056-0078-0219-0437-0630-0705',
    '0010-0017-0030-0040-0042-0053-0056-0078-0219-0437-0630-0705-1017-1195-1439-1532-1828-2032-2196-2380-2482-2516',

]
val_exclude_from_qa_ = 'exclude from QA'
col_exclude_from_qa = 'exclude_from_qa'
df_cluster_for_humans[col_exclude_from_qa] = np.where(
    df_cluster_for_humans[col_new_cluster_val].isin(l_mature_clusters_to_exclude_from_qa),
    val_exclude_from_qa_,
    'keep',  
)

### Add the flag to exclude from QA & the list of sub names to df-raw

In [ ]:
df_labels_target_dynamic_raw = df_labels_target_dynamic_raw.merge(
    df_cluster_for_humans[[col_new_cluster_val, col_list_cluster_names, col_subs_in_cluster_count, col_exclude_from_qa]],
    how='left',
    on=[col_new_cluster_val],
)

df_labels_target_dynamic_raw.shape

(1419, 127)

In [ ]:
df_labels_target_dynamic_raw.iloc[:5, -9:]

,k_2830_topic_mix_nested,k_3145_topic_mix_nested,k_3411_topic_mix_nested,k_3706_topic_mix_nested,k_3864_topic_mix_nested,k_3927_topic_mix_nested,list_cluster_subreddit_names,subs_in_cluster_count,exclude_from_qa
0,Mature Themes and Adult Content | Celebrity,Mature Themes and Adult Content | Celebrity,Mature Themes and Adult Content | Celebrity,Mature Themes and Adult Content | Celebrity,Mature Themes and Adult Content | Celebrity,Mature Themes and Adult Content | Celebrity,"neighboursbabez, stephclairesmith, isabelleclarke, sophadophaa, eugeniebouchard",5,exclude from QA
1,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,"altladyboners, rtgirls, neighboursbabes, aussiebabes, downundercelebs, girlstennis",6,exclude from QA
2,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,"altladyboners, rtgirls, neighboursbabes, aussiebabes, downundercelebs, girlstennis",6,exclude from QA
3,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,"altladyboners, rtgirls, neighboursbabes, aussiebabes, downundercelebs, girlstennis",6,exclude from QA
4,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,Mature Themes and Adult Content,"altladyboners, rtgirls, neighboursbabes, aussiebabes, downundercelebs, girlstennis",6,exclude from QA


## Create new df_clean 

- Add list of subreddits to target-CLEAN, b/c we'll need it for rating final
- Add new columns & update order


### Create some columns & copy baseline cols for clean


In [ ]:
df_labels_target_dynamic_raw[col_new_cluster_val].head().str[-4:].astype(int)

0    1
1    2
2    2
3    2
4    2
Name: cluster_label, dtype: int64

In [ ]:
col_model_sort_order = 'model_sort_order'

# create new col as int for label so we can add a color scale when doing QA
col_new_cluster_val_int = 'cluster_label_int'
df_labels_target_dynamic_raw[col_new_cluster_val_int] = (
    df_labels_target_dynamic_raw[col_new_cluster_val]
    .str[-4:].astype(int)
)

# create new column for reddit link, makes QA easier
col_link_to_sub = 'link_to_sub'
link_prefix_ = 'www.reddit.com/r/'
df_labels_target_dynamic_raw[col_link_to_sub] = link_prefix_ + df_labels_target_dynamic_raw['subreddit_name']

# create lookup for rating col, might need to refresh it later in google sheets, though
col_rated_e = 'rated E'
# need to make it boolean for checkbox thing to work
df_labels_target_dynamic_raw[col_rated_e] = (
    df_labels_target_dynamic_raw['rating_short'] == 'E'
)

# Set expected col order
l_cols_clean_final_for_qa = [
    'subreddit_id',
    'subreddit_name',
    col_new_cluster_val_int,
    col_new_cluster_topic_mix,
    
    # insert inputs for QA cols
    'not country relevant',
    col_rated_e,
    'relevant to cluster/ other subreddits in cluster',
    'safe to show in relation to cluster',
    col_link_to_sub,

    # cols for notes
    'country relevance notes',
    'rating or cluster notes',
    #  # 'cluster relevance notes',

    col_subs_in_cluster_count,
    col_list_cluster_names,

    'allow_discovery',
    'users_l7',
    'total_users_in_subreddit_l28',
    'rating_name',

    'over_18',
    'rating_short',
    'primary_topic',
    col_subreddit_topic_mix,
    # why did a sub get marked as geo or culturally relevant?
    #  can use them to sort
    'geo_relevance_default',
    'relevance_percent_by_subreddit',
    'relevance_percent_by_country_standardized',
    'b_users_percent_by_subreddit',
    'e_users_percent_by_country_standardized',
    'd_users_percent_by_country_rank',
    col_model_sort_order,

    'posts_for_modeling_count',
    'c_users_percent_by_country',
    'users_in_subreddit_from_country_l28',
    'total_users_in_country_l28',
    
    col_new_cluster_val,
    col_new_cluster_name,
    col_new_cluster_prim_topic,
]

# copy existing columns from raw + 
l_cols_clean_existing = [c for c in l_cols_clean_final_for_qa if c in df_labels_target_dynamic_raw.columns]
l_cols_clean_new = [c for c in l_cols_clean_final_for_qa if c not in df_labels_target_dynamic_raw.columns]

# reorder cols in raw DF so it's easier to switch between the two tabs
df_labels_target_dynamic_raw = df_labels_target_dynamic_raw[
    reorder_array(l_cols_clean_existing, df_labels_target_dynamic_raw.columns)
]

# only copy subs that aren't in excluded clusters!!
df_labels_target_dynamic_clean = (
    df_labels_target_dynamic_raw
    [df_labels_target_dynamic_raw[col_exclude_from_qa] != val_exclude_from_qa_]
    [l_cols_clean_existing]
    .copy()
)
# Add new columns and initialize them with empty strings
df_labels_target_dynamic_clean[l_cols_clean_new] = ''

# check num of orphans for clean
display(style_df_numeric(
    get_dynamic_cluster_summary(
        df_labels_target_dynamic_clean,
        return_dict=False,
    ),
    rename_cols_for_display=True,
).set_caption(f"Summary for df-CLEAN"))

# re-order & rename the columns so their easier to see in google sheets
# Sorty by cluster label b/c sometimes a sub won't be clustered dynamically next to closest neighbors!
df_labels_target_dynamic_clean = (
    df_labels_target_dynamic_clean[l_cols_clean_final_for_qa]
    .sort_values(by=[col_new_cluster_val, col_model_sort_order], ascending=True)
    # do final renaming only when saving, otherwise, it's a pain to adjust or look things up?
    .rename(columns={c: c.replace('_', ' ') for c in l_cols_clean_final_for_qa[5:]})
)

print(f"{len(l_cols_clean_final_for_qa)} <- expected final col count")
print(f"{df_labels_target_dynamic_raw.shape} <- df raw shape")
print(f"{df_labels_target_dynamic_clean.shape} <- df clean shape")

,cluster count,num orphan subreddits,num subreddits per cluster mean,num subreddits per cluster median,num clusters with mature primary topic,cluster ids with orphans
0,172,0,7.59,7.00,1,


35 <- expected final col count
(1419, 130) <- df raw shape
(1305, 35) <- df clean shape


In [ ]:
df_labels_target_dynamic_clean.iloc[:4, :]

,subreddit_id,subreddit_name,cluster_label_int,cluster_topic_mix,not country relevant,rated E,relevant to cluster/ other subreddits in cluster,safe to show in relation to cluster,link to sub,country relevance notes,rating or cluster notest,subs in cluster count,list cluster subreddit names,allow discovery,users l7,total users in subreddit l28,rating name,over 18,rating short,primary topic,subreddit full topic mix,geo relevance default,relevance percent by subreddit,relevance percent by country standardized,b users percent by subreddit,e users percent by country standardized,d users percent by country rank,model sort order,posts for modeling count,c users percent by country,users in subreddit from country l28,total users in country l28,cluster label,cluster label k,cluster majority primary topic
134,t5_2syt8,v8supercars,6,Technology,,True,,,www.reddit.com/r/v8supercars,,,6,"v8supercars, discountcodesau, hardwareswapaustralia, gelblastermarket, lockpicking, locksmith",None,2587,5734,Everyone,None,E,Cars and Motor Vehicles,Cars and Motor Vehicles | Sports,True,True,True,0.524939,4.709852,5758,18185,244,0.000139,3010,21632112,0006,k_0013_label,Technology
204,t5_40n7rd,discountcodesau,6,Technology,,False,,,www.reddit.com/r/discountcodesau,,,6,"v8supercars, discountcodesau, hardwareswapaustralia, gelblastermarket, lockpicking, locksmith",None,82,365,None,None,None,None,Marketplace and Deals,True,True,False,0.619178,1.995716,31259,19906,9,0.000010,226,21632112,0006,k_0013_label,Technology
205,t5_32wmx,hardwareswapaustralia,6,Technology,,True,,,www.reddit.com/r/hardwareswapaustralia,,,6,"v8supercars, discountcodesau, hardwareswapaustralia, gelblastermarket, lockpicking, locksmith",None,1147,3636,Everyone,None,E,Marketplace and Deals,Marketplace and Deals,True,True,True,0.845710,4.897608,5663,20025,151,0.000142,3075,21632112,0006,k_0013_label,Technology
206,t5_2x8rfp,gelblastermarket,6,Technology,,False,,,www.reddit.com/r/gelblastermarket,,,6,"v8supercars, discountcodesau, hardwareswapaustralia, gelblastermarket, lockpicking, locksmith",None,241,657,Mature,None,M,Hobbies,Marketplace and Deals,True,True,True,0.754947,2.265542,20016,20055,57,0.000023,496,21632112,0006,k_0013_label,Technology


## Create target to target list for FPR example

Use the df-CLEAN for this, even if it's only a test

In [ ]:
%%time

df_target_to_target_list = convert_distance_or_ab_to_list_for_fpr(
    df_labels_target_dynamic_clean.rename(columns={c: c.replace(' ', '_') for c in df_labels_target_dynamic_clean.columns}),
    convert_to_ab=True,
    col_counterpart_count='counterpart_count',
    col_list_cluster_names='list_cluster_subreddit_names',
    col_list_cluster_ids='list_cluster_subreddit_ids',
    l_cols_for_seeds=None,
    l_cols_for_clusters=None,
    col_new_cluster_val=col_new_cluster_val,
    col_new_cluster_name=col_new_cluster_name,
    col_new_cluster_prim_topic=col_new_cluster_prim_topic,
    verbose=False,
)
df_target_to_target_list.shape

  (9628, 9) <- df_ab.shape after removing matches to self
  (1305, 7) <- df_a_to_b.shape
CPU times: user 129 ms, sys: 1.75 ms, total: 131 ms
Wall time: 141 ms


In [ ]:
df_target_to_target_list.iloc[:5, :11]

,subreddit_name_seed,subreddit_id_seed,cluster_label,cluster_label_k,counterpart_count,list_cluster_subreddit_names,list_cluster_subreddit_ids
0,connectwise,t5_2zw1b,0006-0008-0013,k_0041_label,5,"nuraphone, pixelbuds, googlepixel, bapcsalesaustralia, surface","t5_4mj34, t5_3oydh, t5_3aobb, t5_2tevl, t5_2u5h1"
1,googlewifi,t5_3ggya,0006-0008-0013-0017-0017-0021-0022-0026,k_0118_label,11,"nestwifi, talesfromtechsupport, msp, googlehome, carplay, hisense, chromecast, animelab, funimation, thepiratebays, pocketcasts","t5_26mrs3, t5_2sfg5, t5_2slwq, t5_3enp4, t5_30p7d, t5_3i6jn, t5_2xz02, t5_34h5t, t5_310un, t5_2oc91v, t5_34fxd"
2,nestwifi,t5_26mrs3,0006-0008-0013-0017-0017-0021-0022-0026,k_0118_label,11,"googlewifi, talesfromtechsupport, msp, googlehome, carplay, hisense, chromecast, animelab, funimation, thepiratebays, pocketcasts","t5_3ggya, t5_2sfg5, t5_2slwq, t5_3enp4, t5_30p7d, t5_3i6jn, t5_2xz02, t5_34h5t, t5_310un, t5_2oc91v, t5_34fxd"
3,talesfromtechsupport,t5_2sfg5,0006-0008-0013-0017-0017-0021-0022-0026,k_0118_label,11,"googlewifi, nestwifi, msp, googlehome, carplay, hisense, chromecast, animelab, funimation, thepiratebays, pocketcasts","t5_3ggya, t5_26mrs3, t5_2slwq, t5_3enp4, t5_30p7d, t5_3i6jn, t5_2xz02, t5_34h5t, t5_310un, t5_2oc91v, t5_34fxd"
4,msp,t5_2slwq,0006-0008-0013-0017-0017-0021-0022-0026,k_0118_label,11,"googlewifi, nestwifi, talesfromtechsupport, googlehome, carplay, hisense, chromecast, animelab, funimation, thepiratebays, pocketcasts","t5_3ggya, t5_26mrs3, t5_2sfg5, t5_3enp4, t5_30p7d, t5_3i6jn, t5_2xz02, t5_34h5t, t5_310un, t5_2oc91v, t5_34fxd"


# Export data

Note that we have to use `fillna('')`

Otherwise, we'll get errors because the gspread library doesn't know how to handle `pd.NaN` or `np.Nan` (nulls).

## Define variables to create/access google sheet doc & worksheets

In [ ]:
gspread.__version__

'5.1.1'

In [ ]:
# # %%time
os.environ['GOOGLE_CLOUD_PROJECT'] = 'data-prod-165221'
country_name_sheet_ = 'Australia'

GSHEET_NAME = f'i18n {country_name_sheet_} subreddits and clusters - model v0.4.1'
GSHEET_KEY = '1ujjoJ7uyTWz3P1aYgZTKyJGbETKAfk3vrom_ASaNUt0'
target_abbrev_ = 'au'

d_wsh_names = {
    'qa_ready': {
        'name': 'subs_need_to_be_rated',
    },
    'sub_raw': {
        'name': 'raw_data_per_subreddit',
    },
    'clusters_t2t_list_raw': {
        'name': f'raw_clusters_list_{target_abbrev_}_{target_abbrev_}',
    },
    'clusters_t2t_fpr_raw': {
        'name': f'raw_clusters_fpr_{target_abbrev_}_{target_abbrev_}',
    }
}

if GSHEET_KEY is not None:
    sh = gc.open_by_key(GSHEET_KEY)
    print(f"Opening google worksheet: {GSHEET_NAME} ...")
else:
    print(f"** Creating google worksheet: {GSHEET_NAME} ...")
    sh = gc.create(GSHEET_NAME)

# create worksheets:
for _, d_ in d_wsh_names.items():
    sh_name = d_['name']
    try:
        d_['worksheet'] = sh.worksheet(sh_name)
        print(f"Opening tab/sheet: {sh_name} ...")
    except Exception as e:
        print(f"** Creating tab/sheet: {sh_name} ...")
        d_['worksheet'] = sh.add_worksheet(sh_name, rows=5, cols=5)


Opening google worksheet: i18n Australia subreddits and clusters - model v0.4.1 ...
Opening tab/sheet: subs_need_to_be_rated ...
Opening tab/sheet: raw_data_per_subreddit ...
Opening tab/sheet: raw_clusters_list_au_au ...
Opening tab/sheet: raw_clusters_fpr_au_au ...


## Save: Clean sheet to rate

In [ ]:
d_wsh_names.keys()

dict_keys(['qa_ready', 'sub_raw', 'clusters_t2t_list_raw', 'clusters_t2t_fpr_raw'])

In [ ]:
%%time
(
    d_wsh_names['qa_ready']['worksheet']
    .update([df_labels_target_dynamic_clean.columns.values.tolist()] + 
             df_labels_target_dynamic_clean.fillna('').values.tolist())
)

CPU times: user 51.5 ms, sys: 3.19 ms, total: 54.7 ms
Wall time: 1.65 s


{'spreadsheetId': '1ujjoJ7uyTWz3P1aYgZTKyJGbETKAfk3vrom_ASaNUt0',
 'updatedCells': 45710,
 'updatedColumns': 35,
 'updatedRange': 'subs_need_to_be_rated!A1:AI1306',
 'updatedRows': 1306}

## Save: df cluster for humans

In [ ]:
%%time

(
    d_wsh_names['clusters_t2t_list_raw']['worksheet']
    .update(
        [df_cluster_for_humans.columns.values.tolist()] + 
        df_cluster_for_humans.fillna('').values.tolist()
    )
)

CPU times: user 11.9 ms, sys: 0 ns, total: 11.9 ms
Wall time: 643 ms


{'spreadsheetId': '1ujjoJ7uyTWz3P1aYgZTKyJGbETKAfk3vrom_ASaNUt0',
 'updatedCells': 1330,
 'updatedColumns': 7,
 'updatedRange': 'raw_clusters_list_au_au!A1:G190',
 'updatedRows': 190}

## Save: target raw dynamic


In [ ]:
l_cols_to_drop = (
    ['table_creation_date'] +
    [c for c in df_labels_target_dynamic_raw.columns if c.endswith('_nested')]
)
print(len(l_cols_to_drop))
# df_labels_target_dynamic_raw.columns.to_list()

45


In [ ]:
%%time
(
    d_wsh_names['sub_raw']['worksheet']
    .update([df_labels_target_dynamic_raw.drop(l_cols_to_drop, axis=1).columns.values.tolist()] + 
             df_labels_target_dynamic_raw.drop(l_cols_to_drop, axis=1).fillna('').values.tolist())
)

CPU times: user 120 ms, sys: 5.06 ms, total: 125 ms
Wall time: 4.1 s


{'spreadsheetId': '1ujjoJ7uyTWz3P1aYgZTKyJGbETKAfk3vrom_ASaNUt0',
 'updatedCells': 120700,
 'updatedColumns': 85,
 'updatedRange': 'raw_data_per_subreddit!A1:CG1420',
 'updatedRows': 1420}

## Save: FPR target-2-target as list

Even though data isn't fully ready, want to have the output ready to make sure it's in the right format that we need.

In [ ]:
df_target_to_target_list.iloc[:5, :8]

,subreddit_name_seed,subreddit_id_seed,cluster_label,cluster_label_k,counterpart_count,list_cluster_subreddit_names,list_cluster_subreddit_ids
0,connectwise,t5_2zw1b,0006-0008-0013,k_0041_label,5,"nuraphone, pixelbuds, googlepixel, bapcsalesaustralia, surface","t5_4mj34, t5_3oydh, t5_3aobb, t5_2tevl, t5_2u5h1"
1,googlewifi,t5_3ggya,0006-0008-0013-0017-0017-0021-0022-0026,k_0118_label,11,"nestwifi, talesfromtechsupport, msp, googlehome, carplay, hisense, chromecast, animelab, funimation, thepiratebays, pocketcasts","t5_26mrs3, t5_2sfg5, t5_2slwq, t5_3enp4, t5_30p7d, t5_3i6jn, t5_2xz02, t5_34h5t, t5_310un, t5_2oc91v, t5_34fxd"
2,nestwifi,t5_26mrs3,0006-0008-0013-0017-0017-0021-0022-0026,k_0118_label,11,"googlewifi, talesfromtechsupport, msp, googlehome, carplay, hisense, chromecast, animelab, funimation, thepiratebays, pocketcasts","t5_3ggya, t5_2sfg5, t5_2slwq, t5_3enp4, t5_30p7d, t5_3i6jn, t5_2xz02, t5_34h5t, t5_310un, t5_2oc91v, t5_34fxd"
3,talesfromtechsupport,t5_2sfg5,0006-0008-0013-0017-0017-0021-0022-0026,k_0118_label,11,"googlewifi, nestwifi, msp, googlehome, carplay, hisense, chromecast, animelab, funimation, thepiratebays, pocketcasts","t5_3ggya, t5_26mrs3, t5_2slwq, t5_3enp4, t5_30p7d, t5_3i6jn, t5_2xz02, t5_34h5t, t5_310un, t5_2oc91v, t5_34fxd"
4,msp,t5_2slwq,0006-0008-0013-0017-0017-0021-0022-0026,k_0118_label,11,"googlewifi, nestwifi, talesfromtechsupport, googlehome, carplay, hisense, chromecast, animelab, funimation, thepiratebays, pocketcasts","t5_3ggya, t5_26mrs3, t5_2sfg5, t5_3enp4, t5_30p7d, t5_3i6jn, t5_2xz02, t5_34h5t, t5_310un, t5_2oc91v, t5_34fxd"


In [ ]:
%%time

(
    d_wsh_names['clusters_t2t_fpr_raw']['worksheet']
    .update(
        [df_target_to_target_list.columns.values.tolist()] + 
        df_target_to_target_list.fillna('').values.tolist()
    )
)

CPU times: user 24 ms, sys: 972 µs, total: 24.9 ms
Wall time: 1.14 s


{'spreadsheetId': '1ujjoJ7uyTWz3P1aYgZTKyJGbETKAfk3vrom_ASaNUt0',
 'updatedCells': 9142,
 'updatedColumns': 7,
 'updatedRange': 'raw_clusters_fpr_au_au!A1:G1306',
 'updatedRows': 1306}

### We can read the data back to confirm it's as expected

In [ ]:
# Here's how to get the records as a dataframe
pd.DataFrame(
    d_wsh_names['sub_raw']['worksheet'].get_all_records()
).iloc[:5, :15]

,subreddit_id,subreddit_name,cluster_label_int,cluster_topic_mix,rated E,link_to_sub,subs_in_cluster_count,list_cluster_subreddit_names,allow_discovery,users_l7,total_users_in_subreddit_l28,rating_name,over_18,rating_short,primary_topic
0,t5_44faux,neighboursbabez,1,Mature Themes and Adult Content,FALSE,www.reddit.com/r/neighboursbabez,5,"neighboursbabez, stephclairesmith, isabelleclarke, sophadophaa, eugeniebouchard",,2140,6977,,,,Mature Themes and Adult Content
1,t5_iqt8v,altladyboners,2,Mature Themes and Adult Content,TRUE,www.reddit.com/r/altladyboners,6,"altladyboners, rtgirls, neighboursbabes, aussiebabes, downundercelebs, girlstennis",t,5521,21452,Everyone,f,E,Celebrity
2,t5_3g9c8,rtgirls,2,Mature Themes and Adult Content,TRUE,www.reddit.com/r/rtgirls,6,"altladyboners, rtgirls, neighboursbabes, aussiebabes, downundercelebs, girlstennis",,3134,10235,Everyone,,E,Podcasts and Streamers
3,t5_2bfy1d,neighboursbabes,2,Mature Themes and Adult Content,TRUE,www.reddit.com/r/neighboursbabes,6,"altladyboners, rtgirls, neighboursbabes, aussiebabes, downundercelebs, girlstennis",,3944,14519,Everyone,f,E,Mature Themes and Adult Content
4,t5_39o6d,aussiebabes,2,Mature Themes and Adult Content,TRUE,www.reddit.com/r/aussiebabes,6,"altladyboners, rtgirls, neighboursbabes, aussiebabes, downundercelebs, girlstennis",,945,3820,Everyone,,E,Mature Themes and Adult Content


# Appendix


## Additional checks on cluster depth

In [ ]:
print(df_labels_target_dynamic_raw['cluster_label'].nunique())
display(
    value_counts_and_pcts(
        df_labels_target_dynamic_raw,
        ['cluster_label'],
        top_n=10,
    )
)
value_counts_and_pcts(
    df_labels_target_dynamic_raw,
    ['cluster_label'],
    top_n=None,
    return_df=True
)['count'].describe()

189


,count,percent,cumulative_percent
cluster_label,,,
0007-0011-0019-0026-0027-0036-0037-0047-0136-0274-0404-0454-0659-0777-0934-0994-1201-1336-1439-1566-1639-1664,22,1.5%,1.5%
0008-0013-0023-0031-0033-0043-0045-0062,18,1.3%,2.8%
0007-0012-0020-0027-0028-0037-0038-0048-0138-0277-0409-0459-0669-0790-0950-1011-1219-1358-1462-1593-1668-1693,17,1.2%,4.0%
0010-0016-0029-0039-0041-0052-0055-0077-0217-0433-0625-0699-1010-1187-1430-1521-1815-2018-2181-2364-2464-2498,16,1.1%,5.1%
0006-0010-0018-0025-0025-0033-0034-0043,16,1.1%,6.3%
0010-0017-0030-0040-0043-0054-0057-0082-0227-0451-0651-0728-1054-1239-1489-1585-1895-2104-2272-2466-2572-2607,14,1.0%,7.3%
0006-0010-0017-0023-0023-0031-0032-0041,14,1.0%,8.2%
0013-0021-0037-0054-0057-0072-0076-0106-0287,13,0.9%,9.2%
0013-0021-0037-0054-0057-0072-0077-0107-0289-0587-0852-0948-1381-1634-1953-2081-2503-2771-3001,12,0.8%,10.0%


count    189.000000
mean       7.513228
std        2.400373
min        3.000000
25%        6.000000
50%        7.000000
75%        8.000000
max       22.000000
Name: count, dtype: float64

### How deep are the clusters?

Looks like some peaks around 100, 300, 1k, and 4k clusters.

In [ ]:
print(df_labels_target_dynamic_raw[col_new_cluster_name].nunique())
value_counts_and_pcts(
    df_labels_target_dynamic_raw,
    [col_new_cluster_name],
    top_n=None,
    sort_index=True,
)

22


,count,percent,cumulative_percent
cluster_label_k,,,
k_0013_label,37,2.6%,2.6%
k_0023_label,35,2.5%,5.1%
k_0041_label,30,2.1%,7.2%
k_0059_label,16,1.1%,8.3%
k_0063_label,15,1.1%,9.4%
k_0079_label,12,0.8%,10.2%
k_0085_label,38,2.7%,12.9%
k_0118_label,276,19.4%,32.3%
k_0320_label,194,13.7%,46.0%


In [ ]:
# style_df_numeric(
#     df_labels_target.tail(10),
#     # rename_cols_for_display=True,
#     l_bar_simple=[c for c in df_labels_target.columns if '_label' in c]
# )

# Filter out subs [extra]

UPDATE: For now let's include all the subreddits for QA because this list could potentially help us rate/flag subreddits that aren't rated or mis-rated and have a lot of traffic.

Now that we have even more clusters (over 3,000), it's harder to figure out where to set the threshold for clusters to exclude.

--- 

The main use case for now are SFW subs, we could save some QA time by excluding these subs:
- Exclude NSFW clusters
- Exclude place subs


~We'll use the cluster labels to discard subreddits because~
- many of the DE subreddits don't have a `primary_topic`
- if the majority of subs for a subreddits are NSFW, then we wouldn't want to recommend those anyway

In [ ]:
# # we can see that the subreddit count changes as we go 
# #  from shallow to deeper cluster counts
# value_counts_and_pcts(
#     df_labels_target['k_0118_majority_primary_topic'],
#     top_n=15,
#     reset_index=True,
#     add_col_prefix=False,
#     count_type='subreddits',
#     return_df=False,
# )

In [ ]:
# value_counts_and_pcts(
#     df_labels_target['k_3145_majority_primary_topic'],
#     top_n=15,
#     reset_index=True,
#     add_col_prefix=False,
#     count_type='subreddits',
#     return_df=False,
# )

In [ ]:
# value_counts_and_pcts(
#     df_labels_target['k_3927_majority_primary_topic'],
#     top_n=15,
#     reset_index=True,
#     add_col_prefix=False,
#     count_type='subreddits',
#     return_df=False,
# )

In [ ]:
# # And the count is slightly different from the known primary topics
# #  We still have a large number of subs w/o a primary topic
# value_counts_and_pcts(
#     df_labels_target['primary_topic'],
#     count_type='subreddits',
#     reset_index=True,
#     add_col_prefix=False,
# )

In [ ]:
print(f"{df_labels_target.shape} <- Shape before filtering")

l_manual_subs_to_remove = [
    'sexmeets1', 'fuck',
]
col_cluster_filter = 'k_3145_majority_primary_topic'
df_labels_target_clean = (
    df_labels_target[df_labels_target[col_cluster_filter] != 'Mature Themes and Adult Content']
)
print(f"{df_labels_target_clean.shape} <- Shape after dropping NSFW clusters")

l_sensitive_topics = [
    'Military', 'Gender', 'Addiction Support',
    'Medical and Mental Health', 'Sexual Orientation',
    'Culture, Race, and Ethnicity',
]
df_labels_target_clean = (
    df_labels_target_clean[
        ~df_labels_target_clean[col_cluster_filter].isin(l_sensitive_topics)
    ]
)
print(f"{df_labels_target_clean.shape} <- Shape after dropping Sensitive clusters")

df_labels_target_clean = (
    df_labels_target_clean[
        ~df_labels_target_clean['primary_topic'].isin(l_sensitive_topics)
    ]
)
print(f"{df_labels_target_clean.shape} <- Shape after dropping SENSITIVE subreddits")


df_labels_target_clean = (
    df_labels_target_clean[
        ~df_labels_target_clean['subreddit_name'].isin(l_manual_subs_to_remove)
    ]
)
print(f"{df_labels_target_clean.shape} <- Shape after dropping Manual list of subreddits")

print(f"  ** TODO: instead of excluding place subs, add logic to map hierarchy **")
# df_labels_target_clean = (
#     df_labels_target_clean[df_labels_target_clean['primary_topic'] != 'Place']
# )
# print(f"{df_labels_target_clean.shape} <- Shape after dropping Place subreddits")

(1420, 73) <- Shape before filtering
(1344, 73) <- Shape after dropping NSFW clusters
(1253, 73) <- Shape after dropping Sensitive clusters
(1238, 73) <- Shape after dropping SENSITIVE subreddits
(1238, 73) <- Shape after dropping Manual list of subreddits
  ** TODO: instead of excluding place subs, add logic to map hierarchy **
